# PRÁCTICA 9: PERCEPTRÓN SIMPLE Y ADALINE

###### Técnicas de Aprendizaje Automático 2019/2020 <br>Patricia Aguado Labrador

## EJERCICIO 2

Realizar una regresión lineal aplicando Adaline sobre el dataset 'Boston' con 5000 iteraciones como máximo y una tolerancia de 10<sup>-6</sup>.<br>Aplicar validación cruzada de 10 particiones y calcular tasa de acierto para un error relativo menor del 15% en las intancias de test y el R<sup>2</sup> para cada partición, así como la media y desviación de ambos estadísticos.

In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

#### Descripción del conjunto de datos:

Cargamos el conjunto de datos "boston", el cual contiene información sobre 506 viviendas situadas en el área de Boston, Estados Unidos. El dataset contiene 13 atributos numéricos (tasa de criminalidad, concentración de óxidos nítricos, etc) y una clase numérica que indica el valor medio de las viviendas ocupadas en miles de dolares.

In [2]:
data = load_boston()
y = data.target
# Normalizar
X = MinMaxScaler().fit_transform(data.data)

In [3]:
# Creamos el regresor lineal que utiliza la técnica del descenso de gradiente estocástico a la hora de entrenar
reg = SGDRegressor(max_iter=5000, tol=1e-6, random_state=20)

In [4]:
# Matriz que almacernará los resultados de la tasa de acierto y del R^2 para cada partición
results = pd.DataFrame(columns=['Aciertos','R^2'], index=['Fold 0','Fold 1','Fold 2','Fold 3','Fold 4','Fold 5','Fold 6','Fold 7','Fold 8','Fold 9'])
k = 0
# Creación de 10 particiones para realizar validación cruzada
kf = KFold(n_splits=10,random_state=20)

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Entrenamos el regresor
    reg.fit(X_train, y_train)
    # Predecimos utilizando las intancias de test
    y_predict = reg.predict(X_test)
    
    # Calculamos el error relativo para cada partición
    y_rel_error = abs(y_test-y_predict)/y_test
    
    # Calculamos la tasa de acierto sobre las muestras de test
    # Dividimos el número de instancias que obtienen un error relativo menor del 15% entre el total de muestras de test
    ac = np.argwhere(y_rel_error<0.15).shape[0]/y_rel_error.shape[0]
    # Calculamos el coeficiente de determinación
    r2 = reg.score(X_test,y_test)
    
    results.loc['Fold '+str(k),:] = [ac,r2]
    k +=1

results

,Aciertos,R^2
Fold 0,0.705882,0.762768
Fold 1,0.764706,0.649132
Fold 2,0.745098,-0.132922
Fold 3,0.588235,0.584242
Fold 4,0.607843,0.57645
Fold 5,0.72549,0.713092
Fold 6,0.74,0.501043
Fold 7,0.22,0.0934735
Fold 8,0.28,-0.47204
Fold 9,0.62,0.424197


Calculamos el error relativo dividiendo el valor absoluto de la diferencia entre el valor real y el predicho, entre el valor real.<br>

In [8]:
results1 = pd.DataFrame(columns=['Media','Desviación estándar'], index=['Aciertos','R^2'])
results1.loc['Aciertos',:] = [results.mean().values[0],results.std().values[0]]
results1.loc['R^2',:] = [results.mean().values[1],results.std().values[1]]
results1

,Media,Desviación estándar
Aciertos,0.599725,0.194976
R^2,0.369943,0.407833


Podemos ver que los resultados no son muy buenos ya que la tasa de aciertos de media no es muy alta (los errores relativos que utilizamos para medir la precisión no tienden a ser menores del 15%) y además de media el coeficiente de determinación es bajo aunque variable por lo que refleja la desviación estándar, por lo que nos informa de que la bondad de ajuste del modelo de media no es muy buena. Todo esto nos indica que hay variables que se han recogido que no guardan relación con el precio medio de una vivienda en Boston.